In [7]:
using Revise
using Pkg;
Pkg.activate("../") 
Pkg.instantiate()
using statisticalCTMRG
using TensorKit
using KrylovKit

  Activating project at `c:\Users\psire\work\phd-projects\statisticalCTMRG`


In [44]:
a, b = 0.5, 0.8
D = 15
pattern_arr = Array{Any}(undef,1,1)
pattern_arr[1,1] = 1
T_local_Z2 = generate_local_tensor_Z2(a,b)

free_energy, trunc_arr, env_arr = ctmrg([T_local_Z2], D, pattern_arr; maxiter = 1000, ϵ = 1e-8 , Space_type = :Z2, Projector_type = :half, 
                            conv_info = true, svd_type = :envbond, adjust_χ = false, temp = 1, lognorm = 0, ftol = 10^-10,
                            reuse_envs = false);

CTM-RG iteration 1
the largest truncation error for the projectors is 2.356080457693621e-8
the smallest truncation error for the projectors is 0.0
the relative difference of the real part of the free energies is 1.0
CTM-RG iteration 2
the largest truncation error for the projectors is 1.4901161193847656e-8
the smallest truncation error for the projectors is 1.0536712127723509e-8
the relative difference of the real part of the free energies is 0.0007112951088806577
CTM-RG iteration 3
the largest truncation error for the projectors is 2.9802322387695312e-8
the smallest truncation error for the projectors is 0.0
largest SV-difference of the C tensors is unconverged
the relative difference of the real part of the free energies is 7.085315633805941e-5
CTM-RG iteration 4
the largest truncation error for the projectors is 3.0043490460002353e-7
the smallest truncation error for the projectors is 2.2696772533249583e-7
largest SV-difference of the C tensors is 0.0026078865947539182
the relative 

[ Info: CTMRG did not converge after maxiter = 1000 steps.


### Check sectors of converged environment tensors

In [45]:
@show codomain(env_arr[1].ul), domain(env_arr[1].ul)
@show codomain(env_arr[1].r), domain(env_arr[1].r)

nothing

(codomain((env_arr[1]).ul), domain((env_arr[1]).ul)) = (⊗(Rep[ℤ₂](0 => 7, 1 => 8)), ⊗(Rep[ℤ₂](0 => 8, 1 => 7)))
(codomain((env_arr[1]).r), domain((env_arr[1]).r)) = ((Rep[ℤ₂](0 => 1, 1 => 1) ⊗ Rep[ℤ₂](0 => 7, 1 => 8)), ⊗(Rep[ℤ₂](0 => 7, 1 => 8)))


Populate both sectors for initial boundary tensors (ones)

In [41]:
a, b = 2, 1.7

pattern_arr = Array{Any}(undef,1,1)
pattern_arr[1,1] = 1
D = 50
free_energy, trunc_arr, env_arr = ctmrg([generate_local_tensor_Z2(a,b)], D, pattern_arr; maxiter = 7000, ϵ = 1e-8 , Space_type = :Z2, Projector_type = :half, 
                            conv_info = true, svd_type = :trial2, adjust_χ = false, temp = 1, lognorm = 0, ftol = 10^-10, trunc_check= true,
                            reuse_envs = false);

CTM-RG iteration 1
the largest truncation error for the projectors is 2.1073424255447017e-8
the smallest truncation error for the projectors is 0.0
the relative difference of the real part of the free energies is 1.0
CTM-RG iteration 2
the largest truncation error for the projectors is 1.4901161193847656e-8
the smallest truncation error for the projectors is 0.0
the relative difference of the real part of the free energies is 0.012724864351976811
CTM-RG iteration 3
the largest truncation error for the projectors is 2.5809568279517847e-8
the smallest truncation error for the projectors is 0.0
largest SV-difference of the C tensors is unconverged
the relative difference of the real part of the free energies is 0.004909993942222619
CTM-RG iteration 4
the largest truncation error for the projectors is 2.1073424255447017e-8
the smallest truncation error for the projectors is 0.0
largest SV-difference of the C tensors is unconverged
the relative difference of the real part of the free energi

In [42]:
trunc_arr

7000-element Vector{Float64}:
 2.1073424255447017e-8
 1.4901161193847656e-8
 2.5809568279517847e-8
 2.1073424255447017e-8
 3.650024149988857e-8
 4.470348358154297e-8
 4.9421560620597e-8
 2.9802322387695312e-8
 3.332000937312528e-8
 4.470348358154297e-8
 5.1619136559035694e-8
 4.712160915387242e-8
 5.1619136559035694e-8
 ⋮
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [43]:
function tn_mul_4(X, env_arr)
    @tensor Y[-1, -2; -3] := env_arr.u[-1, 1, 2] * env_arr.d[-2, 3, 1] *
                             X[2, 3, -3]
    return Y
end


V_Z2_even = Z2Space(0 => 1, 1 => 0) 
X = randn(space(env_arr[1].u, 1) ⊗ space(env_arr[1].d, 1),  V_Z2_even)
spectrum_even, _ = eigsolve(X, 12, :LM) do x return tn_mul_4(x, env_arr[1]) end

V_Z2_odd = Z2Space(0 => 0, 1 => 1) 
X = randn(space(env_arr[1].u, 1) ⊗ space(env_arr[1].d, 1),  V_Z2_odd)
spectrum_odd, _ = eigsolve(X, 12, :LM) do x return tn_mul_4(x, env_arr[1]) end

V_Z2 = Z2Space(0 => 1, 1 => 1) 
X = randn(space(env_arr[1].u, 1) ⊗ space(env_arr[1].d, 1),  V_Z2)
spectrum_all, _ = eigsolve(X, 12, :LM) do x return tn_mul_4(x, env_arr[1]) end

@show spectrum_even
@show spectrum_odd
@show spectrum_all

nothing

spectrum_even = ComplexF64[1.0269449536210995 + 0.0im, -1.0269449535788273 + 0.0im, -0.750610222453731 + 0.0im, 0.7506102224242126 + 0.0im, -0.7365561082093771 + 0.0im, 0.7365561081765721 + 0.0im, -0.7132943643695516 + 0.0im, 0.7132943643260569 + 0.0im, -0.6811116541517886 + 0.0im, 0.681111654102758 + 0.0im, -0.6404971287344086 + 0.0im, 0.6404971286925761 + 0.0im]
spectrum_odd = ComplexF64[1.0269449536210986 + 0.0im, -1.0269449535788302 + 0.0im, -0.7506102224537353 + 0.0im, 0.7506102224242113 + 0.0im, -0.7365561082093737 + 0.0im, 0.7365561081765697 + 0.0im, -0.7132943643695568 + 0.0im, 0.713294364326062 + 0.0im, -0.6811116541517941 + 0.0im, 0.6811116541027522 + 0.0im, -0.6404971287344066 + 0.0im, 0.6404971286925757 + 0.0im]
spectrum_all = ComplexF64[1.0269449536211008 + 0.0im, 1.0269449536210993 + 0.0im, -1.0269449535788235 + 0.0im, -1.026944953578822 + 0.0im, -0.7506102224537391 + 0.0im, -0.7506102224537327 + 0.0im, 0.7506102224242205 + 0.0im, 0.7506102224242202 + 0.0im, -0.7365561082